In [1]:
import mlflow
import keras
import numpy as np
import pandas as pd
from mlflow.models import infer_signature
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import train_test_split

In [2]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

In [3]:
mlflow.set_experiment("MLflow Run Comparison Quickstart")

<Experiment: artifact_location='mlflow-artifacts:/510851129115464979', creation_time=1740954005594, experiment_id='510851129115464979', last_update_time=1740954005594, lifecycle_stage='active', name='MLflow Run Comparison Quickstart', tags={}>

In [4]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

In [5]:
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()

In [6]:
train_x, valid_x, train_y, valid_y = train_test_split(train_x,
                                                      train_y,
                                                      test_size=0.2,
                                                      random_state=42)

In [7]:
signature = infer_signature(train_x, train_y)

In [8]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y):
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )

        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [9]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y
    )
    return result

In [10]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}

In [11]:
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 11s 250ms/step - loss: 37.4799 - root_mean_squared_error: 6.1221
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.7078 - root_mean_squared_error: 2.4045 - val_loss: 0.9833 - val_root_mean_squared_error: 0.9916

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.6959 - root_mean_squared_error: 0.8342
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8170 - root_mean_squared_error: 0.9037 - val_loss: 0.6731 - val_root_mean_squared_error: 0.8205

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.4757 - root_mean_squared_error: 0.6897
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6239 - root_mean_squared_error: 0.7895 - val_loss: 0.5875 - val_root_mean_squared_error: 0.7665

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.4931 - root_mean_squared_error: 0.7022
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step